In [1]:
import pandas as pd
import build_db
from core.data import dbclient
from core.data import data_transformations

In [2]:
# connect to db and show all tables
db = dbclient.DBClient()
db.cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(db.cursor.fetchall())

[('VACCINATIONS',), ('DAILY_COVID_CASE_DATA',), ('DAILY_FOOT_TRAFFIC_DATA',), ('TRAFFIC_CRASH_DATA',), ('DEMOGRAPHICS',)]


### Daily Case Data

In [3]:
query = f"select * from {build_db.CASE_TBL}"
print(f"query = {query}")
covid_case_df = pd.read_sql_query(query, db.conn)
covid_case_df.describe(include='all')

query = select * from DAILY_COVID_CASE_DATA


,index,STD_DATE,ZIPCODE,confirmed_cases,confirmed_cases_change,total_tested,total_tested_change,AVG7DAY_confirmed_cases,AVG7DAY_confirmed_cases_change,AVG7DAY_total_tested
count,333098.000000,333098,332995,333098,333098,333098,333098,324332.000000,324332.000000,324332.000000
unique,NaN,323,1446,6895,195,39670,1776,NaN,NaN,NaN
top,NaN,2021-03-12 00:00:00,60098,6,0,147,0,NaN,NaN,NaN
freq,NaN,1430,323,9014,149887,346,28360,NaN,NaN,NaN
mean,166548.500000,NaN,NaN,NaN,NaN,NaN,NaN,478.640146,3.558449,6376.932092
std,96157.254321,NaN,NaN,NaN,NaN,NaN,NaN,999.274745,7.379647,18170.056928
min,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,6.000000,-6.571429,10.428571
25%,83274.250000,NaN,NaN,NaN,NaN,NaN,NaN,26.000000,0.142857,387.000000
50%,166548.500000,NaN,NaN,NaN,NaN,NaN,NaN,99.571429,0.857143,1232.285714
75%,249822.750000,NaN,NaN,NaN,NaN,NaN,NaN,447.857143,3.428571,5750.607143


### Vaccination Data

In [4]:
query = f"select * from {build_db.VACC_TBL}"
print(f"query = {query}")
vacc_df = pd.read_sql_query(query, db.conn)
vacc_df.describe(include='all')

query = select * from VACCINATIONS


,index,ZIPCODE,STD_DATE,total_doses_daily,total_doses_cumulative,vaccine_series_completed_daily,vaccine_series_completed_percent_population,population,AVG7DAY_total_doses_daily,AVG7DAY_vaccine_series_completed_daily
count,5000.000000,4919,5000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,4565.000000,4565.000000
unique,NaN,59,91,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,60634,2021-03-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,91,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2499.500000,NaN,NaN,143.768600,4530.254600,51.510400,0.033289,46211.641800,146.053294,52.594148
std,1443.520003,NaN,NaN,162.413316,5334.012868,72.468759,0.046259,26625.453119,134.711408,60.168219
min,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1249.750000,NaN,NaN,22.000000,516.000000,0.000000,0.000000,28569.000000,38.285714,3.714286
50%,2499.500000,NaN,NaN,89.500000,2275.000000,20.000000,0.014000,46591.000000,110.714286,30.000000
75%,3749.250000,NaN,NaN,214.250000,7000.750000,73.000000,0.049000,67711.000000,217.857143,80.142857


### Join Cases and Vaccinations

In [8]:
query = (f"select case_data.{data_transformations.STD_ZIP_COL_NAME},"
         f" case_data.{data_transformations.STD_DATE_COL_NAME},"
         f" vacc_data.{data_transformations.STD_ZIP_COL_NAME} ZIPB,"
         f" vacc_data.{data_transformations.STD_DATE_COL_NAME} DATEB,"
         f" case_data.AVG7DAY_confirmed_cases,"
         f" case_data.AVG7DAY_confirmed_cases_change,"
         f" vacc_data.AVG7DAY_total_doses_daily, vacc_data.AVG7DAY_vaccine_series_completed_daily"
         f" from {build_db.CASE_TBL} case_data left join {build_db.VACC_TBL} vacc_data"
         f" on case_data.{data_transformations.STD_ZIP_COL_NAME} = vacc_data.{data_transformations.STD_ZIP_COL_NAME}"
         f" and case_data.{data_transformations.STD_DATE_COL_NAME} = vacc_data.{data_transformations.STD_DATE_COL_NAME}"
         f" where case_data.{data_transformations.STD_ZIP_COL_NAME} in (select distinct )"")

case_and_vacc_df = pd.read_sql_query(query, db.conn)
case_and_vacc_df['AVG7DAY_total_doses_daily'].fillna(0, inplace=True)
case_and_vacc_df['AVG7DAY_vaccine_series_completed_daily'].fillna(0, inplace=True)

print(case_and_vacc_df.sort_values(by='AVG7DAY_vaccine_series_completed_daily'))

       ZIPCODE             STD_DATE   ZIPB                DATEB  \
0         None  2020-04-18 00:00:00   None                 None   
219610   61602  2020-12-24 00:00:00   None                 None   
219609   61603  2020-12-24 00:00:00   None                 None   
219608   61604  2020-12-24 00:00:00   None                 None   
219607   61605  2020-12-24 00:00:00   None                 None   
...        ...                  ...    ...                  ...   
331069   60629  2021-03-14 00:00:00  60629  2021-03-14 00:00:00   
329640   60629  2021-03-13 00:00:00  60629  2021-03-13 00:00:00   
329651   60618  2021-03-13 00:00:00  60618  2021-03-13 00:00:00   
328209   60629  2021-03-12 00:00:00  60629  2021-03-12 00:00:00   
331080   60618  2021-03-14 00:00:00  60618  2021-03-14 00:00:00   

        AVG7DAY_confirmed_cases  AVG7DAY_confirmed_cases_change  \
0                           NaN                             NaN   
219610                83.142857                        1.5714

In [6]:
print(query)
print()
print(f"nrow covid cases: {len(covid_case_df)}")
print(f"nrow vaccinations: {len(vacc_df)}")
print(f"nrow left joined data: {len(case_and_vacc_df)}")
print(f"nrow left joined data vacc NOT null: {len(case_and_vacc_df) - case_and_vacc_df['ZIPB'].isna().sum()}")
print(f"num null covid cases: {case_and_vacc_df['AVG7DAY_confirmed_cases'].isna().sum()}")
print()

pd.set_option('display.max_rows', 500)
print(case_and_vacc_df.sort_values(by='AVG7DAY_vaccine_series_completed_daily'))
case_and_vacc_df.describe(include='all')

select case_data.ZIPCODE, case_data.STD_DATE, vacc_data.ZIPCODE ZIPB, vacc_data.STD_DATE DATEB, case_data.AVG7DAY_confirmed_cases, case_data.AVG7DAY_confirmed_cases_change, vacc_data.AVG7DAY_total_doses_daily, vacc_data.AVG7DAY_vaccine_series_completed_daily from DAILY_COVID_CASE_DATA case_data left join VACCINATIONS vacc_data on case_data.ZIPCODE = vacc_data.ZIPCODE and case_data.STD_DATE = vacc_data.STD_DATE join VACCINATIONS on case_data.ZIPCODE = VACCINATIONS.ZIPCODE

nrow covid cases: 333098
nrow vaccinations: 5000
nrow left joined data: 1559211
nrow left joined data vacc NOT null: 395826
num null covid cases: 29028

        ZIPCODE             STD_DATE   ZIPB                DATEB  \
0         60642  2020-04-18 00:00:00   None                 None   
834047    60608  2020-10-13 00:00:00   None                 None   
834046    60608  2020-10-13 00:00:00   None                 None   
834045    60608  2020-10-13 00:00:00   None                 None   
834044    60608  2020-10-13 00

,ZIPCODE,STD_DATE,ZIPB,DATEB,AVG7DAY_confirmed_cases,AVG7DAY_confirmed_cases_change,AVG7DAY_total_doses_daily,AVG7DAY_vaccine_series_completed_daily
count,1559211,1559211,395826,395826,1.530183e+06,1.530183e+06,1.559211e+06,1.559211e+06
unique,58,323,58,89,NaN,NaN,NaN,NaN
top,60634,2021-01-04 00:00:00,60604,2021-03-03 00:00:00,NaN,NaN,NaN,NaN
freq,29393,4838,8099,4838,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,2.064525e+03,1.310084e+01,3.516283e+01,1.273505e+01
std,NaN,NaN,NaN,NaN,2.330360e+03,1.667455e+01,9.145641e+01,3.741698e+01
min,NaN,NaN,NaN,NaN,6.142857e+00,-1.857143e+00,0.000000e+00,0.000000e+00
25%,NaN,NaN,NaN,NaN,5.614286e+02,3.285714e+00,0.000000e+00,0.000000e+00
50%,NaN,NaN,NaN,NaN,1.311429e+03,7.285714e+00,0.000000e+00,0.000000e+00
75%,NaN,NaN,NaN,NaN,2.723571e+03,1.628571e+01,0.000000e+00,0.000000e+00
